### The notebook is adapted from https://juliaacademy.com/

## Classification
[Classification](https://www.datacamp.com/blog/classification-machine-learning) is a supervised machine learning process that involves predicting the class of given data points.  Those classes can be targets, labels or categories. Put simply, classification is the task of predicting a label for a given observation. For example: you are given certain physical descriptions of an animal, and your taks is to classify them as either a dog or a cat. Here, we will classify iris flowers.

As we will see later, we will use different classifiers and at the end of this notebook, we will compare them. We will define our accuracy function right now to get it out of the way. We will use a simple accuracy function that returns the ratio of the number of correctly classified observations to the total number of predictions.

Common classification algorithms include: K-nearest neighbor, decision trees, naive Bayes and artificial neural networks.

In [1]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [2]:
import Pkg; Pkg.add("GLMNet")
Pkg.add("RDatasets")
Pkg.add("MLBase")
Pkg.add("Plots")
Pkg.add("DecisionTree")
Pkg.add("Distances")
Pkg.add("NearestNeighbors")
Pkg.add("Random")
Pkg.add("LinearAlgebra")
Pkg.add("DataStructures")
Pkg.add("LIBSVM")

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


...

  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [3]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

Get the data first

In [4]:
iris = dataset("datasets", "iris")

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [5]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

150-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [6]:
X

150×4 Matrix{Float64}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 4.7  3.2  1.3  0.2
 4.6  3.1  1.5  0.2
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.9  3.1  1.5  0.1
 ⋮              
 6.9  3.1  5.1  2.3
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3
 5.9  3.0  5.1  1.8

`X` is a matrix that contains the features of the iris dataset. The iris dataset consists of measurements of 150 iris flowers from three different species. There are four features measured: sepal length, sepal width, petal length, and petal width. By doing `X = Matrix(iris[:,1:4])`, we are extracting these four features from the dataset and storing them in `X`.

The reason for checking or examining `X` is to understand the structure and properties of your data before applying machine learning algorithms. This could involve checking the size of `X`, looking at the range of values for each feature, or visualizing the data. Understanding your data can help you choose appropriate machine learning methods and interpret the results correctly.

In the code cell below, `irislabelsmap = labelmap(irislabels)` is creating a mapping of the labels in the data to integer values. The `labelmap` function from the `MLBase` package in Julia creates a dictionary that maps each unique value in `irislabels` to a unique integer.

Then, `y = labelencode(irislabelsmap, irislabels)` is using this mapping to transform the labels into integers. The `labelencode` function takes the mapping created by `labelmap` and the original labels, and returns a vector where each original label is replaced by its corresponding integer.

This is often done in machine learning when you have categorical labels. Many machine learning algorithms require numerical input and output, so it's common to encode categorical labels as integers.

In [10]:
# Create a mapping of the labels to integers
irislabelsmap = labelmap(irislabels)
# irislabelsmap is now a dictionary that maps each unique label in irislabels to a unique integer

# Use this mapping to transform our labels into integers
y = labelencode(irislabelsmap, irislabels)
# y now contains integer labels corresponding to the original labels in irislabels

150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3

In classification, we often want to use some of the data to fit a model, and the rest of the data to validate (commonly known as `training` and `testing` data). We will get this data ready now so that we can easily use it in the rest of this notebook.

In [11]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

The `randsubseq` function is a built-in Julia function that returns a random subsequence of a given sequence. It’s used in `perclass_splits` function to select a random subset of indices for each class.

In [13]:
randsubseq

randsubseq (generic function with 2 methods)

In [14]:
# trainids are the indices of the training data
trainids = perclass_splits(y,0.7)

# testids are the indices of the test data
testids = setdiff(1:length(y),trainids)

35-element Vector{Any}:
   6
   7
  13
  20
  22
  23
  26
  27
  33
  37
   ⋮
 114
 117
 118
 124
 133
 140
 145
 148
 149

We will need one more function, and that is the function that will assign classes based on the predicted values when the predicted values are continuous. This is necessary when the predicted values are continuous but the labels are discrete. The function works by finding the class label that is closest to the predicted value.

In [15]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

### 🟣 Method 1: Lasso
It's a type of regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the statistical model it produces. In Julia, you can use the `glmnet` function from the `GLMNet` package to fit a generalized linear model via penalized maximum likelihood, which is the principle behind the Lasso method¹.

In your code, `path = glmnet(X[trainids,:], y[trainids])` is fitting a Lasso model to your training data. The `glmnet` function computes a path of solutions for varying penalty strength, which is useful for tuning the Lasso model².

Then, `cv = glmnetcv(X[trainids,:], y[trainids])` is performing cross-validation to find the optimal penalty strength³. Cross-validation is a resampling procedure used to evaluate machine learning models on a limited data sample. The procedure has a single parameter called k that refers to the number of groups that a given data sample is to be split into.

References:
1. Lasso.jl - JuliaStats. https://juliastats.org/Lasso.jl/stable/
2. GitHub - JuliaStats/Lasso.jl: Lasso/Elastic Net linear and generalized. https://github.com/JuliaStats/Lasso.jl
3. Introduction to Lasso Regression - Statology. https://www.statology.org/lasso-regression/

In [16]:
# Fit a Lasso model to the training data
path = glmnet(X[trainids,:], y[trainids])

# Perform cross-validation to find the optimal penalty strength
cv = glmnetcv(X[trainids,:], y[trainids])

Least Squares GLMNet Cross Validation
72 models for 4 predictors in 10 folds
Best λ 0.002 (mean loss 0.054, std 0.007)

The output indicates that cross-validation was performed with 72 models for 4 predictors in 10 folds. The best lambda (λ) value found was 0.002, which resulted in a mean loss of 0.054 and a standard deviation of 0.007.

This way, you're preparing your data correctly for the machine learning algorithms you'll use later on! 😊

In [17]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda]);

In [18]:
q = X[testids,:];

# The predict function takes the fitted model and the features of the testing data, and returns predicted values.
predictions_lasso = GLMNet.predict(path,q)

35×1 Matrix{Float64}:
 1.0980319067139004
 1.023125473670924
 0.9268306739701447
 1.010077648257915
 1.0801925512521762
 0.8807193545134296
 1.022107433474087
 1.1184022331748664
 0.8625253412214363
 0.9367683918978442
 ⋮
 2.7412172522271927
 2.660230836988896
 3.047726581736285
 2.5847452674598523
 2.9422876256488446
 2.8250433630010314
 3.11029450502679
 2.7370891045450465
 2.928660663749768

In [19]:
# The predicted values are continuous but the labels are discrete
# assign a class label to each predicted value
predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso,y[testids])

1.0

The result suggests that the model is effectively capturing patterns in the data and could be used to make reliable predictions on new data.

### 🟣 Method 2: Ridge
Ridge regression is a method for improving the conditioning of the problem and reducing the variance of the estimates. In Ridge regression, the penalty term is the L2 norm (the square root of the sum of the squares of the coefficients), while in Lasso regression, it’s the L1 norm (the sum of the absolute values of the coefficients). This leads to different properties for each method¹.

Ridge regression is a method of estimating the coefficients of multiple-regression models in scenarios where the independent variables are highly correlated¹. It is particularly useful to mitigate the problem of multicollinearity in linear regression, which commonly occurs in models with large numbers of parameters¹.

In Ridge regression, a shrinkage penalty is introduced to the sum of squared residuals (RSS) which is minimized in ordinary least squares regression². This penalty term is controlled by a parameter lambda (λ), and it has the effect of shrinking the estimated coefficients towards zero². When λ = 0, Ridge regression produces the same coefficient estimates as least squares. However, as λ approaches infinity, the shrinkage penalty becomes more influential and the Ridge regression coefficient estimates approach zero².

The advantage of Ridge regression compared to least squares regression lies in the bias-variance tradeoff². The basic idea of Ridge regression is to introduce a little bias so that the variance can be substantially reduced, which leads to a lower overall mean squared error (MSE)².

Ridge regression is often used when the number of predictor variables in a set exceeds the number of observations, or when a data set has multicollinearity (correlations between predictor variables)³. It has been used in many fields including econometrics, chemistry, and engineering¹.

References:
1. Ridge regression - Wikipedia. https://en.wikipedia.org/wiki/Ridge_regression
2. Introduction to Ridge Regression - Statology. https://www.statology.org/ridge-regression/
3. Ridge Regression: Simple Definition - Statistics How To. https://www.statisticshowto.com/ridge-regression/
4. Ridge Regression Definition & Examples | What is Ridge Regression?. https://www.mygreatlearning.com/blog/what-is-ridge-regression/

### Exercise 1

We will use the same function but set alpha to zero.

Tips: 
- Check the previous three cell
- Add `alpha=0` in the `glmnet` and `glmnetcv` functions. This sets the elastic net mixing parameter to 0, which corresponds to Ridge regression.

In [20]:
# Choose the best lambda to predict with


1.0

### 🟣 Method 3: Elastic Net

**Elastic Net** is a regularized regression method that linearly combines the L1 and L2 penalties of the Lasso and Ridge methods¹. It is particularly useful when there are multiple correlated features²³.

In more detail, Elastic Net is a compromise between Lasso regression (which uses an L1 penalty) and Ridge regression (which uses an L2 penalty). The Elastic Net algorithm uses both L1 and L2 penalties, effectively combining properties of both Lasso and Ridge¹².

The key advantage of Elastic Net over Lasso is that it can select groups of correlated variables, whereas Lasso tends to select one variable from each group and ignore the others¹. This makes Elastic Net particularly useful when dealing with high-dimensional data where predictors are highly correlated.

References:
1. Elastic net regularization - Wikipedia. https://en.wikipedia.org/wiki/Elastic_net_regularization
2. Scikit Learn - Elastic-Net - Online Tutorials Library. https://www.tutorialspoint.com/scikit_learn/scikit_learn_elastic_net.htm
3. Elastic Net Regression Explained, Step by Step - Machine Learning Compass. https://machinelearningcompass.com/machine_learning_models/elastic_net_regression/
4. Elastic Net - Overview, Geometry, and Regularization. https://corporatefinanceinstitute.com/resources/data-science/elastic-net/

The Elastic Net method includes the LASSO and Ridge regression: in other words, each of them is a special case where α=1 or α=0, respectively. Here, α is the mixing parameter between Ridge (α=0) and Lasso (α=1). If α=0.5, it's an equal mix of Ridge and Lasso¹.
We will use the same function as before but set alpha to 0.5 (it's the combination of lasso and ridge).
We fit the model to the training data, perform cross-validation to find the optimal penalty strength, make predictions on the testing data, and then calculate the accuracy of the predictions.

In [21]:
# Choose the best lambda to predict with
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

1.0

### 🟣 Method 4: Decision Trees
**Decision Trees** are a type of non-parametric supervised learning method used for both classification and regression tasks¹². The goal of a decision tree is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features³⁴. Detailed explanation:

1. **Structure**: A decision tree is composed of nodes, which include root nodes, internal nodes, and leaf nodes¹. A root node has outgoing edges but no incoming edges. An internal node represents a feature in our data which is used to split the data into two or more children nodes. At each internal node, a decision is made based on the feature to determine which child node to proceed to. A leaf node represents the outcome (or class label in classification tasks) and contains no outgoing edges¹.

2. **Decision Making**: The decisions at each node are made based on certain conditions on the features. For example, if we have a feature 'Age', an internal node could have a condition like 'Age < 18', and the tree would route data instances based on whether this condition is true or false².

3. **Learning**: The process of learning a decision tree from data involves determining the best feature to split on at each node, and this is often done using measures such as information gain or Gini impurity².

4. **Prediction**: To make a prediction for a new instance, you start at the root of the tree and follow the appropriate path through the internal nodes until you reach a leaf node. The prediction of the decision tree is then the value associated with that leaf node².

5. **Advantages**: Decision trees are popular because they're easy to understand and interpret, can handle both numerical and categorical data, and can handle multi-output problems².

6. **Disadvantages**: They can create over-complex trees that overfit the data and can be unstable because small variations in the data might result in a completely different tree being generated².

References:
1. Decision Tree Algorithm - TowardsMachineLearning. https://towardsmachinelearning.org/decision-tree-algorithm/
2. Decision Trees — scikit-learn 1.3.1 documentation. https://scikit-learn.org/stable/modules/tree.html
3. Decision tree - Wikipedia. https://en.wikipedia.org/wiki/Decision_tree
4. Decision tree learning - Wikipedia. https://en.wikipedia.org/wiki/Decision_tree_learning
5. Decision Tree - Overview, Decision Types, Applications. https://corporatefinanceinstitute.com/resources/data-science/decision-tree/

We will use the package `DecisionTree`. 

In [22]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [23]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

1.0

### 🟣 Method 5: Random Forests
**Random Forests** is a powerful machine learning technique that's used for both regression and classification problems¹²³. It's an ensemble learning method, which means it combines the predictions of multiple machine learning algorithms to make more accurate predictions⁴. Detailed explanation:

1. **Ensemble of Decision Trees**: A Random Forest is essentially a collection of decision trees, each trained on a random subset of the data¹²⁴. Each tree in the forest is built from a sample drawn with replacement (i.e., a bootstrap sample) from the training set³.

2. **Feature Randomness**: In addition, when splitting each node during the construction of a tree, the best split is found either from all input features or a random subset of size `m` (where `m` is specified by the user)³.

3. **Prediction**: For a classification problem, the output of the Random Forest is the class selected by most trees (majority vote). For regression problems, it's typically the average prediction of all trees¹².

4. **Advantages**: Random Forests correct for decision trees' habit of overfitting to their training set¹. They generally outperform decision trees and are robust to outliers and non-linear data¹². They also handle large datasets with high dimensionality well⁵.

5. **Disadvantages**: However, they're not easily interpretable and can be computationally intensive depending on the number of trees and depth².

References:
1. Random forest - Wikipedia. https://en.wikipedia.org/wiki/Random_forest.
2. What is Random Forest? | IBM. https://www.ibm.com/topics/random-forest
3. Introduction to Random Forest in Machine Learning. https://www.section.io/engineering-education/introduction-to-random-forest-in-machine-learning/
4. Random Forest - Overview, Modeling Predictions, Advantages. https://corporatefinanceinstitute.com/resources/data-science/random-forest/
5. Random Forests – SpringerLink. https://link.springer.com/article/10.1023/A:1010933404324

### Exercise 2

Use Random Forests in a similar way as you used Ridge, Lasso, Elastic Net, and Decision Trees. 

Tips: 
- Check the previous cells
- The `RandomForestClassifier` is available through the `DecisionTree` package as well
- n_trees=20

The `n_trees` parameter in the Random Forest algorithm refers to the number of trees that will be grown in the forest. Each of these trees will be trained on a different subset of the data and will make its own predictions. The final prediction of the Random Forest is then based on the majority vote (for classification) or average (for regression) of the predictions of all the trees.

The choice of `n_trees=20` is somewhat arbitrary and is often chosen based on empirical results. In general, having more trees in the forest can increase the model's performance because it allows for more diverse subsets of the data to be considered. However, after a certain point, adding more trees can lead to diminishing returns in terms of accuracy improvement, and can also increase computational cost.

It's often a good idea to experiment with different values of `n_trees` and use cross-validation to find the value that gives the best performance on your specific dataset. In practice, values in the range of 100-1000 are commonly used, but smaller values like 20 can also give good results depending on the dataset and problem at hand. 😊

In [25]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

1.0

### 🟣 Method 6: Using a Nearest Neighbor method
Nearest Neighbor method is a type of proximity search optimization problem that aims to find the point in a given set that is closest or most similar to a given point. This method is often used in machine learning for both classification and regression tasks. Nearest Neighbor method for classification is method based on the principle that instances within a dataset will generally exist in close proximity to other instances that have similar properties¹.

First, we're creating a KDTree, a space-partitioning data structure for organizing points in a K-dimensional space, with the training data using `kdtree = KDTree(Xtrain')`. The KDTree will allow efficient spatial queries for the nearest neighbors².

Next, we're defining the queries, which are the data points you want to classify, with `queries = X[testids,:]`.

Then, we're finding the 5 nearest neighbors of each query point in the KDTree with `idxs, dists = knn(kdtree, queries', 5, true)`. The `knn` function returns two arrays: `idxs` contains the indices of the 5 nearest neighbors of each query point, and `dists` contains the distances to these neighbors².

After that, we're getting the labels of the nearest neighbors with `c = ytrain[hcat(idxs...)]`.

Then, we're counting the number of occurrences of each label in the nearest neighbors of each query point with `possible_labels = map(i->counter(c[:,i]),1:size(c,2))`. The `counter` function returns a dictionary where the keys are the unique labels and the values are their counts².

Next, we're assigning each query point to the class that occurs most frequently among its 5 nearest neighbors with `predictions_NN = map(i->parse(Int,string(string(argmax(possible_labels[i])))),1:size(c,2))`. If there's a tie for the most frequent class, `argmax` will return one of them at random².

Finally, we're calculating the accuracy of your predictions with `findaccuracy(predictions_NN,y[testids])`. The output shows that the model achieved an accuracy of some % on the testing data. 

References:
1. Neighborhood.jl: Unified API for finding nearest neighbors in Julia. https://discourse.julialang.org/t/neighborhood-jl-unified-api-for-finding-nearest-neighbors-in-julia/38659
2. NearestNeighbors · Julia Packages. https://juliapackages.com/p/nearestneighbors
3. GitHub - KristofferC/NearestNeighbors.jl: High performance nearest. https://github.com/KristofferC/NearestNeighbors.jl
4. GitHub - JuliaAI/NearestNeighborModels.jl: Package providing K-nearest. https://github.com/JuliaAI/NearestNeighborModels.jl

In [26]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArraysCore.SVector{4, Float64}, Euclidean, Float64}
  Number of points: 115
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [27]:
queries = X[testids,:]

35×4 Matrix{Float64}:
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 4.8  3.0  1.4  0.1
 5.1  3.8  1.5  0.3
 5.1  3.7  1.5  0.4
 4.6  3.6  1.0  0.2
 5.0  3.0  1.6  0.2
 5.0  3.4  1.6  0.4
 5.2  4.1  1.5  0.1
 5.5  3.5  1.3  0.2
 ⋮              
 5.7  2.5  5.0  2.0
 6.5  3.0  5.5  1.8
 7.7  3.8  6.7  2.2
 6.3  2.7  4.9  1.8
 6.4  2.8  5.6  2.2
 6.9  3.1  5.4  2.1
 6.7  3.3  5.7  2.5
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3

In [28]:
idxs, dists = knn(kdtree, queries', 5, true)

([[16, 9, 35, 33, 14], [3, 10, 22, 4, 28], [2, 8, 34, 26, 23], [35, 5, 15, 1, 20], [15, 5, 35, 1, 20], [3, 28, 31, 5, 27], [26, 8, 2, 23, 34], [18, 6, 30, 15, 10], [25, 35, 9, 14, 5], [9, 24, 21, 35, 20]  …  [99, 114, 80, 107, 103], [111, 93, 88, 63, 115], [107, 80, 99, 87, 86], [102, 81, 85, 105, 94], [97, 114, 98, 54, 103], [99, 80, 87, 109, 89], [87, 113, 110, 92, 109], [109, 92, 112, 95, 106], [86, 113, 89, 107, 58], [106, 89, 86, 109, 107]], [[0.33166247903553986, 0.3464101615137753, 0.3605551275463989, 0.3741657386773947, 0.3999999999999999], [0.264575131106459, 0.3000000000000002, 0.31622776601683805, 0.33166247903553986, 0.4123105625617666], [0.1414213562373099, 0.17320508075688815, 0.19999999999999998, 0.20000000000000037, 0.244948974278318], [0.2449489742783178, 0.26457513110645875, 0.31622776601683783, 0.33166247903553986, 0.33166247903553997], [0.244948974278318, 0.26457513110645897, 0.2828427124746191, 0.30000000000000016, 0.3000000000000003], [0.5099019513592785, 0.509901

In [29]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(string(argmax(possible_labels[i])))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

1.0

### 🟣 Method 7: Support Vector Machines
**Support Vector Machines (SVM)** is a supervised machine learning model that uses classification algorithms for two-group classification problems¹. After giving an SVM model sets of labeled training data for each category, they're able to categorize new text². Detailed explanation:

1. **Classification and Regression**: SVMs are a set of supervised learning methods used for classification, regression, and outliers detection.

2. **High Dimensionality**: They are effective in high dimensional spaces and still effective in cases where the number of dimensions is greater than the number of samples².

3. **Memory Efficiency**: SVMs use a subset of training points in the decision function (called support vectors), so it is also memory efficient².

4. **Versatility**: Different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

5. **Disadvantages**: If the number of features is much greater than the number of samples, avoiding over-fitting in choosing Kernel functions and regularization term is crucial. SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation⁴.

References:
1. Support Vector Machines (SVM) Algorithm Explained - MonkeyLearn. https://monkeylearn.com/blog/introduction-to-support-vector-machines-svm/
2. Support Vector Machines — scikit-learn 1.3.1 documentation. https://scikit-learn.org/stable/modules/svm.html
3. Support vector machine - Wikipedia. https://en.wikipedia.org/wiki/Support_vector_machine
4. Introduction to Support Vector Machines (SVM) - GeeksforGeeks. https://www.geeksforgeeks.org/introduction-to-support-vector-machines-svm/

In [30]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

115-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [31]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64, LIBSVM.Kernel.KERNEL}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 115, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Vector{Int64}, Matrix{Float64}}(42, Int32[7, 18, 17], [1, 1, 1, 1, 1, 1, 1, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4.3 5.7 … 6.3 5.9; 3.0 4.4 … 2.5 3.0; 1.1 1.5 … 5.0 5.1; 0.1 0.4 … 1.9 1.8], Int32[11, 13, 16, 18, 19, 32, 33, 38, 39, 40  …  98, 100, 102, 103, 104, 108, 110, 111, 114, 115], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 4.3), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 4.8), LIBSVM.SVMNode(1, 4.5), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.7)  …  LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 7.2), LIBSVM.SVMNode(1, 7.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 6.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.0 0.010690561607055334; 0.445434185

In [32]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

1.0

Putting all the results together:

In [33]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

7×2 Matrix{Any}:
 "lasso"  1.0
 "ridge"  1.0
 "EN"     1.0
 "DT"     1.0
 "RF"     1.0
 "kNN"    1.0
 "SVM"    1.0

# Finally...
After finishing this notebook, you should be able to:
- [ ] split your data into training and testing data to test the effectiveness of a certain method
- [ ] apply a simple accuracy function to test the effectiveness of a certain method
- [ ] run multiple classification algorithms:
    - [ ] LASSO
    - [ ] Ridge
    - [ ] ElasticNet
    - [ ] Decision Tree
    - [ ] Random Forest
    - [ ] Nearest Neighbors
    - [ ] Support Vector Machines

# 🥳 One cool finding

We used multiple methods to run classification on the `iris` dataset which is a dataset of flowers and there are three types of iris flowers in it. We split the data into training and testing and ran our methods. Here is the scoreboard:

| method | accuracy score |
|---|---|
| lasso  |1.0|
| ridge  |1.0|
| EN     |1.0|
| DT     |0.960784|
| RF     |0.980392|
| kNN    |1.0|
| SVM    |1.0|

In [ ]:
# Solutions
# Exercise 1
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

In [24]:
# Exercise 2
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 7.35
Avg Depth:  4.75